In [1]:
import pathlib
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches
import mplstereonet
import skimage.exposure
import harmonica as hm
import verde as vd

import zipfile
import os
import micromag as mg

import time
import warnings
import rich.progress

In [2]:
# !pip install h5netcdf

In [3]:
import scipy as sp
import numba
import choclo
# @numba.jit(nopython=True, parallel=True)
def goal_function(parameter, obs_data, coordinates, amplitude, x_0, y_0, z_0):
    bz = mg.dipole_bz(coordinates, 
                      ([parameter[0]*x_0], [parameter[1]*y_0], [parameter[2]*z_0]),
                     np.array([parameter[3], parameter[4], parameter[5]])*amplitude
                     )
                   
    
    Res = np.linalg.norm(obs_data-bz.ravel())
    # print(Res)
    return(Res)

In [4]:
# Nome do arquivo ZIP
zip_filename = "simulations_18k_grains_per_mm3.zip"
output_folder = "csv_files_18k_grains_per_mm3"
os.makedirs(output_folder, exist_ok=True)


# Abrir o arquivo ZIP e listar os arquivos dentro dele
with zipfile.ZipFile(zip_filename, "r") as zipf:
    # Listar os nomes dos arquivos NetCDF no ZIP
    netcdf_files = [name for name in zipf.namelist() if name.endswith(".nc")]
    # print(f"Arquivos encontrados no ZIP: {netcdf_files}")
    
    # Iterar sobre todos os arquivos NetCDF no ZIP
    for i, nc_file in enumerate(netcdf_files):
        with zipf.open(nc_file) as file:
            print(f"Running file {i+1}: {nc_file}")
            data = xr.open_dataset(file)
            
            x_, y_ = np.meshgrid(data.x.values, data.y.values)
            coordinates = ([x_, y_ , data.z.values])

            # Run the code
            data_copy = data.copy(deep=True)
            data_copy2 = data_copy.copy(deep=True)

            height_difference = 5
            
            # Have to assign the same points as the data because the Harmonica
            # transforms give slightly different coordinates due to round-off.
            # This is a bug and is being worked on.
            data_up = (
                hm.upward_continuation(data.bz, height_difference)
                .assign_attrs(data.bz.attrs)
                .to_dataset(name="bz")
                .assign_coords(x=data.x, y=data.y)
                .assign_coords(z=data.z + height_difference)
            )
            data_up = data_up.assign(mg.data_gradients(data_up.bz))
            
            stretched = skimage.exposure.rescale_intensity(
                data_up.tga, 
                in_range=tuple(np.percentile(data_up.tga, (1, 99))),
            )
            data_up = data_up.assign(tga_stretched=xr.DataArray(stretched, coords=data_up.coords))


            windows = mg.detect_anomalies(
                data_up.tga_stretched, 
                size_range=[20, 150],
                size_increment=1.0,
                threshold=0.05,
                overlap=0.0,
                exclude_border=15
            )
            
            ## Standard method
            positions = []
            estimated_dipole_moments = []
            estimated_stds = []
            calculated_r2 = []
            calculated_SNR = []
            windows_filtered = []
            base_levels = []
            for window in windows:
                anomaly = data_up.sel(x=slice(*window[:2]), y=slice(*window[2:]))
                position, base_level = mg.euler_deconvolution(
                    anomaly.bz, 
                    anomaly.x_deriv, 
                    anomaly.y_deriv, 
                    anomaly.z_deriv,
                )
                
                data_minus_background = anomaly.bz.values - base_level
                anomaly = anomaly.assign(data_minus_background=(['y','x'], data_minus_background))
                anomaly.data_minus_background.attrs = {"long_name": "dipole-model", "units": "nT"}
            
                moment, covariance, r2, SNR = mg.dipole_moment_inversion(
                    anomaly.data_minus_background, position,
                )
                
                bad_euler = False # position[2] > 0
                poor_fit = False
                
                if bad_euler or poor_fit:
                    continue
                positions.append(position)  
                estimated_dipole_moments.append(moment)
                estimated_stds.append(mg.covariance_to_angle_std(moment, covariance))
                calculated_r2.append(r2)
                calculated_SNR.append(SNR)
                windows_filtered.append(window)
                base_levels.append(base_level)
            positions = np.transpose(positions)
            
            #### IF NEEDED INSERT HERE THE MISFITS CALC FOR THE STANDARD METHOD #######
            ##
            #### IF NEEDED INSERT HERE THE MISFITS CALC FOR THE STANDARD METHOD #######
    
            
            ## Iterative Method
            warnings.filterwarnings("ignore")
            
            x_concat = []
            y_concat = []
            z_concat = []
            bz_concat = []
            xc_concat = []
            yc_concat = []
            zc_concat = []
            z_deriv_concat = []
            
            estimated_dipole_moments_itr_euler = []
            
            # fitting parameters
            calculated_r2_itr_euler = []
            calculated_SNR_itr_euler = []
            
            data_copy = data.copy(deep=True)
            data_up_copy = data_up.copy(deep=True)
            
            #################################
            for window in rich.progress.track(np.array(windows), total = len(np.array(windows))):
                anomaly = data_up_copy.sel(x=slice(*window[:2]), y=slice(*window[2:]))
                position, base_level = mg.euler_deconvolution(
                    anomaly.bz, 
                    anomaly.x_deriv, 
                    anomaly.y_deriv, 
                    anomaly.z_deriv,
                )
                
                data_minus_background = anomaly.bz.values - base_level 
                anomaly = anomaly.assign(data_minus_background=(['y','x'], data_minus_background))
                anomaly.data_minus_background.attrs = {"long_name": "dipole-model", "units": "nT"}
            
                
                moment, covariance, r2, SNR = mg.dipole_moment_inversion(anomaly.data_minus_background, position)
            
                ## SCIPY MINIMIZATION
                anomaly_table=vd.grid_to_table(anomaly)
                amplitude = np.linalg.norm(moment)
                args=(data_minus_background.ravel(), 
                     ( anomaly_table.x.values,
                       anomaly_table.y.values, 
                       anomaly.z.values.ravel()),
                      amplitude, position[0], position[1], position[2])
                
                minimization=sp.optimize.minimize(goal_function, (1, 1, 1,
                                                                  moment[0]/amplitude,
                                                                  moment[1]/amplitude,
                                                                  moment[2]/amplitude),
                                                  
                                                  args=args,
                                                  
                                                    method='Nelder-Mead', options=dict(fatol=1.0e-8))
                
                if minimization.success:
                    
                    moment = np.array([minimization['x'][3], minimization['x'][4], minimization['x'][5]])*amplitude
                    position = np.array([minimization['x'][0]*position[0], 
                                         minimization['x'][1]*position[1],
                                         minimization['x'][2]*position[2]])
                    xxx, yyy = np.meshgrid(anomaly.x.values, anomaly.y.values)
                    zzz = anomaly.z.values.ravel()
                    pred = mg.dipole_bz([xxx, yyy, zzz], 
                              position,
                              moment
                             )
                    
                    residuals = data_minus_background.ravel() - pred.ravel()
                    residuals_sum_sq = np.sum(residuals**2)
                    r2 = 1 - residuals_sum_sq / np.linalg.norm(data_minus_background.ravel() - np.mean(data_minus_background.ravel())) ** 2
                
            
                estimated_dipole_moments_itr_euler.append(moment)
                
                #########
                
                discard = mg.dipole_bz(coordinates, position, moment)
            
                data_copy.bz.values -= discard
                
                data_up_copy = (
                            hm.upward_continuation(data_copy.bz, height_difference)
                            .assign_attrs(data_copy.bz.attrs)
                            .to_dataset(name="bz")
                            .assign_coords(x=data_copy.x, y=data_copy.y)
                            .assign_coords(z=data_copy.z + height_difference)
                               )
                
                
                data_up_copy = data_up_copy.assign(mg.data_gradients(data_up_copy.bz))
                
                
                ########
                bad_euler = False #position[2] > 0.0
                poor_fit =  False # r2 < 0.85  
            
                if bad_euler or poor_fit:
                    continue
                xx, yy = np.meshgrid(anomaly.x.values, anomaly.y.values)
                x_concat = np.append(x_concat, xx)
                y_concat = np.append(y_concat, yy)    
                z_concat = np.append(z_concat, anomaly.z.values)
                bz_concat = np.append(bz_concat, anomaly.bz.values)
                z_deriv_concat = np.append(z_deriv_concat, anomaly.z_deriv.values)
                xc_concat = np.append(xc_concat, position[0])
                yc_concat = np.append(yc_concat, position[1])
                zc_concat = np.append(zc_concat, position[2])
                calculated_r2_itr_euler = np.append(calculated_r2_itr_euler, r2)
                calculated_SNR_itr_euler = np.append(calculated_SNR_itr_euler, SNR)
            
            df = pd.DataFrame({'x':x_concat,
                               'y':y_concat,
                               'z':z_concat,
                              'bz':bz_concat,
                             'z_deriv':z_deriv_concat})
            
            positions_itr = np.array([xc_concat, yc_concat, zc_concat])
            
            
            
            ## EULER ENHANCEMENT SECOND STEP
            warnings.filterwarnings("ignore")
            data_copy = data.copy(deep=True)
            euler_r2_itr_cond = calculated_r2_itr_euler>=0.975
            
            zc_concat_temp = list()
            xc_concat_temp = list()
            yc_concat_temp = list()
            
            estimated_dipole_moments_itr_euler_temp = list()
            
            indexes = np.where(np.array(euler_r2_itr_cond))[0]
            # SOURCES WITH R2 >= 0.99 ARE NOT ENHANCED, WE CALCULATE THEIR ANOMALY AND REMOVE THEM FROM THE DATA SET
            # for index, window in enumerate(rich.progress.track(np.array(windows)[euler_r2_itr_cond], total=len(np.array(windows)[euler_r2_itr_cond]))):
            for index in rich.progress.track(indexes, total=len(indexes)):
                window = windows[index]
                position = positions_itr[:, index]
                estimated_dipole_moments_itr_euler_temp.append(estimated_dipole_moments_itr_euler[index])
                xc_concat_temp = np.append(xc_concat_temp, position[0])
                yc_concat_temp = np.append(yc_concat_temp, position[1])
                zc_concat_temp = np.append(zc_concat_temp, position[2])  
            
                
            # remove all signals 
            position_temp = np.array([xc_concat_temp, yc_concat_temp, zc_concat_temp])
            discard = mg.dipole_bz(coordinates, position_temp, estimated_dipole_moments_itr_euler_temp)
            data_copy.bz.values -= discard
            
            data_up_copy = (
                        hm.upward_continuation(data_copy.bz, height_difference)
                        .assign_attrs(data_copy.bz.attrs)
                        .to_dataset(name="bz")
                        .assign_coords(x=data_copy.x, y=data_copy.y)
                        .assign_coords(z=data_copy.z + height_difference)
                           )
            
            
            data_up_copy = data_up_copy.assign(mg.data_gradients(data_up_copy.bz))
            
            
            # SOURCES WITH R2 <= 0.99 ARE ENHANCED
            indexes = np.where(~np.array(euler_r2_itr_cond))[0]
            # for index, window in enumerate(rich.progress.track(np.array(windows)[~euler_r2_itr_cond], total=len(np.array(windows)[~euler_r2_itr_cond]))):
            for index in rich.progress.track(indexes, total=len(indexes)):
                window = windows[index]
                anomaly = data_up_copy.sel(x=slice(*window[:2]), y=slice(*window[2:]))
                position, base_level = mg.euler_deconvolution(
                    anomaly.bz, 
                    anomaly.x_deriv, 
                    anomaly.y_deriv, 
                    anomaly.z_deriv,
                )
                data_minus_background = anomaly.bz.values - base_level 
                anomaly = anomaly.assign(data_minus_background=(['y','x'], data_minus_background))
                anomaly.data_minus_background.attrs = {"long_name": "dipole-model", "units": "nT"}
                
                moment, covariance, r2, SNR = mg.dipole_moment_inversion(anomaly.data_minus_background, position)
            
            
            
                ## SCIPY MINIMIZATION
                anomaly_table=vd.grid_to_table(anomaly)
                amplitude = np.linalg.norm(moment)
                args=(data_minus_background.ravel(), 
                     ( anomaly_table.x.values,
                       anomaly_table.y.values, 
                       anomaly.z.values.ravel()),
                      amplitude, position[0], position[1], position[2])
                
                minimization=sp.optimize.minimize(goal_function, (1, 1, 1,
                                                                  moment[0]/amplitude,
                                                                  moment[1]/amplitude,
                                                                  moment[2]/amplitude),
                                                  
                                                  args=args,
                                                  
                                                    method='Nelder-Mead', options=dict(fatol=1.0e-8))
                
                if minimization.success:
                    
                    moment = np.array([minimization['x'][3], minimization['x'][4], minimization['x'][5]])*amplitude
                    position = np.array([minimization['x'][0]*position[0], 
                                         minimization['x'][1]*position[1],
                                         minimization['x'][2]*position[2]])
                    xxx, yyy = np.meshgrid(anomaly.x.values, anomaly.y.values)
                    zzz = anomaly.z.values.ravel()
                    pred = mg.dipole_bz([xxx, yyy, zzz], 
                              position,
                              moment
                             )
                    
                    residuals = data_minus_background.ravel() - pred.ravel()
                    residuals_sum_sq = np.sum(residuals**2)
                    r2 = 1 - residuals_sum_sq / np.linalg.norm(data_minus_background.ravel() - np.mean(data_minus_background.ravel())) ** 2
                    
                    if r2 >= calculated_r2_itr_euler[index]:
                        estimated_dipole_moments_itr_euler[index] = moment
                        positions_itr[:, index] = position
                
                discard = mg.dipole_bz(coordinates, position, moment)
            
                data_copy.bz.values -= discard
            
                data_up_copy = (
                            hm.upward_continuation(data_copy.bz, height_difference)
                            .assign_attrs(data_copy.bz.attrs)
                            .to_dataset(name="bz")
                            .assign_coords(x=data_copy.x, y=data_copy.y)
                            .assign_coords(z=data_copy.z + height_difference)
                               )
                
                
                data_up_copy = data_up_copy.assign(mg.data_gradients(data_up_copy.bz))
            
            
            
            #### IF NEEDED INSERT HERE THE MISFITS CALC FOR THE ITERATIVE METHOD #######
            ##
            #### IF NEEDED INSERT HERE THE MISFITS CALC FOR THE ITERATIVE METHOD #######
            
            
            ## RE RUN THE RESIDUAL TO FIND MORE SOURCES
            data_up_copy = (
                        hm.upward_continuation(data_copy.bz, height_difference)
                        .assign_attrs(data_copy.bz.attrs)
                        .to_dataset(name="bz")
                        .assign_coords(x=data_copy.x, y=data_copy.y)
                        .assign_coords(z=data_copy.z + height_difference)
                           )
            
            data_up_copy = data_up_copy.assign(mg.data_gradients(data_up_copy.bz))
            
            stretched = skimage.exposure.rescale_intensity(
                data_up_copy.tga, 
                in_range=tuple(np.percentile(data_up_copy.tga, (1, 99))),
            )
            data_up_copy = data_up_copy.assign(tga_stretched=xr.DataArray(stretched, coords=data_up.coords))
            
            windows_new = mg.detect_anomalies(
                data_up_copy.tga_stretched, 
                size_range=[20,150],
                size_increment=1.3,
                threshold=0.02,
                overlap=0.0,
                exclude_border=15
            )
            
            def intersects(window1, window2):
                # Verifica se há sobreposição horizontal
                horizontal_overlap = (window1[0] <= window2[1]) and (window2[0] <= window1[1])
                # Verifica se há sobreposição vertical
                vertical_overlap = (window1[2] <= window2[3]) and (window2[2] <= window1[3])
                return horizontal_overlap and vertical_overlap
                
            
            # Remove overlapping windows
            non_overlapping_windows = np.copy(windows_new)
            
            for window in windows:
                non_overlapping_windows = [nw for nw in non_overlapping_windows if not intersects(nw, window)]
            
            
            data_copy_2 = data_copy.copy(deep=True)
            data_up_copy_2 = data_up_copy.copy(deep=True)
            
            positions_itr_enhanced = list(np.copy(positions_itr))
            estimated_dipole_moments_itr_euler_enhanced = list(np.copy(estimated_dipole_moments_itr_euler))
            windows_enhanced = list(np.copy(windows))
            calculated_r2_itr_euler_enhanced = list(np.copy(calculated_r2_itr_euler))
            
            #################################
            for window in rich.progress.track(np.array(non_overlapping_windows), total = len(np.array(non_overlapping_windows))):
                anomaly = data_up_copy_2.sel(x=slice(*window[:2]), y=slice(*window[2:]))
                position, base_level = mg.euler_deconvolution(
                    anomaly.bz, 
                    anomaly.x_deriv, 
                    anomaly.y_deriv, 
                    anomaly.z_deriv,
                )
                
                data_minus_background = anomaly.bz.values - base_level 
                anomaly = anomaly.assign(data_minus_background=(['y','x'], data_minus_background))
                anomaly.data_minus_background.attrs = {"long_name": "dipole-model", "units": "nT"}
            
                
                moment, covariance, r2, SNR = mg.dipole_moment_inversion(anomaly.data_minus_background, position)
            
                ## SCIPY MINIMIZATION
                anomaly_table=vd.grid_to_table(anomaly)
                amplitude = np.linalg.norm(moment)
                args=(data_minus_background.ravel(), 
                     ( anomaly_table.x.values,
                       anomaly_table.y.values, 
                       anomaly.z.values.ravel()),
                      amplitude, position[0], position[1], position[2])
                
                minimization=sp.optimize.minimize(goal_function, (1, 1, 1,
                                                                  moment[0]/amplitude,
                                                                  moment[1]/amplitude,
                                                                  moment[2]/amplitude),
                                                  
                                                  args=args,
                                                  
                                                    method='Nelder-Mead', options=dict(fatol=1.0e-8))
                
                if minimization.success:
                    
                    moment = np.array([minimization['x'][3], minimization['x'][4], minimization['x'][5]])*amplitude
                    position = np.array([minimization['x'][0]*position[0], 
                                         minimization['x'][1]*position[1],
                                         minimization['x'][2]*position[2]])
                    xxx, yyy = np.meshgrid(anomaly.x.values, anomaly.y.values)
                    zzz = anomaly.z.values.ravel()
                    pred = mg.dipole_bz([xxx, yyy, zzz], 
                              position,
                              moment
                             )
                    
                    residuals = data_minus_background.ravel() - pred.ravel()
                    residuals_sum_sq = np.sum(residuals**2)
                    r2 = 1 - residuals_sum_sq / np.linalg.norm(data_minus_background.ravel() - np.mean(data_minus_background.ravel())) ** 2
            
            
                windows_enhanced.append(window)
                estimated_dipole_moments_itr_euler_enhanced.append(moment)
                for i in range(3):
                    positions_itr_enhanced[i]=np.append(positions_itr_enhanced[i], position[i])
                calculated_r2_itr_euler_enhanced.append(r2)
                #########
                
                discard = mg.dipole_bz(coordinates, position, moment)
            
                data_copy_2.bz.values -= discard
                
                data_up_copy_2 = (
                            hm.upward_continuation(data_copy_2.bz, height_difference)
                            .assign_attrs(data_copy_2.bz.attrs)
                            .to_dataset(name="bz")
                            .assign_coords(x=data_copy_2.x, y=data_copy_2.y)
                            .assign_coords(z=data_copy_2.z + height_difference)
                               )
                
                
                data_up_copy_2 = data_up_copy_2.assign(mg.data_gradients(data_up_copy_2.bz))
                
                
                ########
                bad_euler = False #position[2] > 0.0
                poor_fit =  False # r2 < 0.85  
            
                if bad_euler or poor_fit:
                    continue
            
            #### IF NEEDED INSERT HERE THE MISFITS CALC FOR THE RE RUN METHOD #######
            ##
            #### IF NEEDED INSERT HERE THE MISFITS CALC FOR THE RE RUN METHOD #######

            # Criar um DataFrame com os vetores
            standard_dataframe = pd.DataFrame({
                'mx_standard': np.asarray(estimated_dipole_moments)[:,0],
                'my_standard': np.asarray(estimated_dipole_moments)[:,1],
                'mz_standard': np.asarray(estimated_dipole_moments)[:,2],
                'r_2_standard': np.asarray(calculated_r2),
            })
            standard_csv_name = os.path.join(output_folder, nc_file.replace('.nc', '_standard.csv'))
            standard_dataframe.to_csv(standard_csv_name, index=False)

            # Criar DataFrame para os vetores iterativos
            iterative_dataframe = pd.DataFrame({
                'mx_iterative': np.asarray(estimated_dipole_moments_itr_euler_enhanced)[:,0],
                'my_iterative': np.asarray(estimated_dipole_moments_itr_euler_enhanced)[:,1],
                'mz_iterative': np.asarray(estimated_dipole_moments_itr_euler_enhanced)[:,2],
                'r_2_iterative': np.asarray(calculated_r2_itr_euler_enhanced),
            })

            # Salvar DataFrame iterativo como CSV
            iterative_csv_name = os.path.join(output_folder, nc_file.replace('.nc', '_iterative.csv'))
            iterative_dataframe.to_csv(iterative_csv_name, index=False)


Running file 1: data_simulation_7.nc


/home/gelson/anaconda3/envs/micromag-euler-dipole/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)
/home/gelson/anaconda3/envs/micromag-euler-dipole/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)
/home/gelson/anaconda3/envs/micromag-euler-dipole/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)


Output()

Output()

Output()

...

Running file 10: data_simulation_5.nc


Output()

Output()

Output()

Output()

In [5]:
np.shape(estimated_dipole_moments)

(300, 3)